In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sbr
from datetime import datetime
import matplotlib.pyplot as plt
import random
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.neural_network import MLPClassifier
from treatment_train import clean_data as clean_train
from treatment_test import clean_data as clean_test

In [2]:
train_df = gpd.read_file('train.geojson')
test_df = gpd.read_file('test.geojson')

In [11]:
train_df['change_type'].value_counts()

Residential      148435
Commercial       100422
Demolition        31509
Road              14305
Industrial         1324
Mega Projects       151
Name: change_type, dtype: int64

In [3]:
train_df_clean, cluster_groups = clean_train(train_df)
test_df_clean, test_df_nan = clean_test(test_df, cluster_groups)

c:\Users\nicol\OneDrive\Documents\DD\SG6\Machine Learning\Kaggle\treatment_train.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_area = np.asarray(df[['geometry']].area).reshape(-1, 1)
c:\Users\nicol\OneDrive\Documents\DD\SG6\Machine Learning\Kaggle\treatment_train.py:113: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  train_per = np.asarray(df[['geometry']].length).reshape(-1, 1)
c:\Users\nicol\OneDrive\Documents\DD\SG6\Machine Learning\Kaggle\treatment_test.py:86: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  test_area = np.asarray(df[['geometry']].area).reshape(-1, 1

In [ ]:
from scipy.stats import loguniform as sp_loguniform
from sklearn.model_selection import RandomizedSearchCV

# Instancia nosso classificador
mlp = MLPClassifier(random_state=42, early_stopping=True)

# Hiperparâmetros a serem otimizados
hyperparams = {'hidden_layer_sizes': [(2 ** i, 2 ** j) for j in np.arange(5, 8) for i in np.arange(4, 7)],
               'alpha': sp_loguniform(1e-7, 1e-1),
               'learning_rate': ['constant','adaptive']}

# Busca de Hiperparâmetros
mlp_search_cv = RandomizedSearchCV(mlp, hyperparams, scoring='accuracy', n_iter=15, cv=5, n_jobs=-1, random_state=42)
%timeit -n 1 -r 1 mlp_search_cv.fit(X_train, Y_train)

print('Melhores hiperparâmetros: {}'.format(mlp_search_cv.best_params_))
print('Desempenho do melhor modelo: {}'.format(round(mlp_search_cv.best_score_,5)))